In [2]:
import numpy as np
import pandas as pd

In [2]:
####version 3 of dataframe,same as 2 but encoding late a quartile, aslso including arr_dely

In [3]:
from sqlalchemy import create_engine
# Postgres username, password, and database name
POSTGRES_ADDRESS = 'lhl-data-bootcamp.crzjul5qln0e.ca-central-1.rds.amazonaws.com' ## INSERT YOUR DB ADDRESS IF IT'S NOT ON PANOPLY
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'lhl_student' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = 'lhl_student' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD
POSTGRES_DBNAME = 'mid_term_project' ## CHANGE THIS TO YOUR DATABASE NAME
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))
# Create the connection
cnx = create_engine(postgres_str)

In [ ]:
full = pd.read_sql_query('''DROP TABLE IF EXISTS flight_subset;
DROP TABLE IF EXISTS passenger_subset;

SELECT AVG(arr_delay) AS avg_arr_delay, op_unique_carrier, AVG(dep_delay) AS avg_dep_delay, CONCAT(flights.origin_airport_id, '-', flights.dest_airport_id) AS route, SUBSTRING(fl_date, 6, 2) AS month, SUBSTRING(fl_date, 1, 4) AS year
INTO TABLE flight_subset
FROM flights
WHERE diverted != 1.0
GROUP BY route, month, year, op_unique_carrier;

ALTER TABLE flight_subset ALTER COLUMN month TYPE BIGINT USING(month::bigint), ALTER COLUMN year TYPE BIGINT USING (year::bigint);

SELECT departures_scheduled, departures_performed, seats, passengers, freight, mail, aircraft_group, aircraft_type, unique_carrier, region, origin_airport_id, dest_airport_id, CONCAT(origin_airport_id, '-', dest_airport_id) AS route, month, year, distance
INTO TABLE passenger_subset
FROM passengers;

SELECT MAX(avg_arr_delay) AS avg_arr_delay, passenger_subset.route, passenger_subset.month, passenger_subset.year, unique_carrier, MAX(avg_dep_delay) AS avg_dep_delay, MAX(departures_scheduled) AS departures_scheduled, MAX(departures_performed) AS departures_performed, MAX(seats) AS seats, MAX(passengers) AS passengers, MAX(freight) AS freight, MAX(mail) AS mail, MAX(aircraft_group) AS aircraft_group, MAX(aircraft_type) AS aircraft_type, MAX(region) AS region, MAX(passenger_subset.origin_airport_id) AS origin_airport_id, MAX(passenger_subset.dest_airport_id) AS dest_airport_id, MAX(distance) AS distance
FROM flight_subset JOIN passenger_subset ON (flight_subset.route = passenger_subset.route AND op_unique_carrier = unique_carrier AND flight_subset.month = passenger_subset.month AND flight_subset.year = passenger_subset.year)
GROUP BY passenger_subset.route, passenger_subset.month, passenger_subset.year, unique_carrier;''',cnx)

In [ ]:
full

In [ ]:
full = full.dropna()

In [ ]:
full

In [ ]:
full.nunique()

In [ ]:
full["avg_arr_delay"].head()

In [ ]:
full

In [ ]:
full["avg_arr_delay"].head()

In [ ]:
full["avg_arr_delay"].quantile(q=[.1, .2, .3, .4, .5, .6, .7, .8, .9, 1])

In [ ]:
full["avg_arr_delay"] = pd.qcut(full["avg_dep_delay"], 10, [0,1,2,3,4,5,6,7,8,9])

In [ ]:
full

In [ ]:
full.isnull().nunique()

In [ ]:
mo = pd.get_dummies(full["month"], prefix = "Month")

In [ ]:
ye =pd.get_dummies(full["year"], prefix = "year")

In [ ]:
opa = pd.get_dummies(full["unique_carrier"], prefix = "unique_carrier")

In [ ]:
ag = pd.get_dummies(full["aircraft_group"], prefix = "aircraft_group")

In [ ]:
at = pd.get_dummies(full["aircraft_type"], prefix = "aircraft_type")

In [ ]:
re = pd.get_dummies(full["region"], prefix = "Region")

In [ ]:
oai = pd.get_dummies(full["origin_airport_id"], prefix = "origin_airport_id")

In [ ]:
dai = pd.get_dummies(full["dest_airport_id"], prefix = "dest_airport_id")

In [ ]:
full = pd.concat([full,mo,ye,opa,ag,at,re,oai,dai], axis=1)

In [ ]:
full

In [ ]:
full = full.drop(columns=["avg_dep_delay","month","year","route", "unique_carrier", "aircraft_group","aircraft_type", "region", "origin_airport_id","dest_airport_id"])

In [ ]:
full

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
# cols = ['avg_dep_delay', 'departures_scheduled', 'departures_performed', 'seats', 'passengers', 'freight', 'mail', 'distance']
# numeric = full[cols].copy()
# numeric = pd.DataFrame(minmax.fit_transform(numeric), columns=cols)
# full.drop(columns=cols, inplace=True)
# full.join(numeric)
# full.drop(columns=cols)

In [ ]:
cols = ['departures_scheduled', 'departures_performed', 'seats', 'passengers', 'freight', 'mail', 'distance']
cols_scaled = [s + "_scaled" for s in cols]
minmax = MinMaxScaler()
full[cols_scaled] = minmax.fit_transform(full[cols])

In [ ]:
full

In [ ]:
full = full.drop(columns=['departures_scheduled', 'departures_performed', 'seats', 'passengers', 'freight', 'mail', 'distance'])

In [ ]:
full

In [ ]:
#full.to_csv("dfversion3.csv", index=False)